## ナップザック問題解答例
ナップザック問題をどのように解くのか事例を知りたいと思った

### pythonライブラリ
- 通常の関数だけでも作れることを知った

In [2]:
n = int(input())
w = []
v = []
for i in range(n):
    x,y = map(int,input().split())
    w.append(x)
    v.append(y)
W = int(input())
dp = [[0]*(W+1) for j in range(n+1)]

for i in range(n-1,-1,-1):
    for j in range(W+1):
        if j < w[i]:
            dp[i][j] = dp[i+1][j]
        else:
            dp[i][j] = max(dp[i+1][j],dp[i+1][j-w[i]]+v[i])
print(dp[0][W])

 4
 2 3
 4


ValueError: not enough values to unpack (expected 2, got 1)

dp[i][j] : i 番目以降の品物から重さが j 以下になるように選んだ時の価値の最大値

dpについて
一番下の段（i = n) は０（n 番目の物など存在しない）

dp[i][j]について
i+1 番目以降の品物の理想の選び方が分かってるとしたら（dp[i+1][~]がわかってるとしたら）
① i 番目の品物はそもそも持てない（ j < w[i]）
　dp[i][j] = dp[i+1][j] ( i +1 番目以降の物のみを使う場合の理想の選び方）

② i 番目の品物を持とうと思えば持てる ( j >= w[i] )
　dp[i][j] は
　⑴ i 番目の品物を持たなかった場合の理想の選び方 ( dp[i+1][j]）
　⑵ i + 1 番目の品物を持った場合の理想の選び方 (dp[i+1][j-w[i]] + v[i]）
　のどちらか良いほうである。
 
 
想定入力例
```
4
2 3
1 2
3 4
2 2
5
```
出力　`7`

### minisat
- `NameError: name 'SatSolver' is not defined`となって使えなかった

In [4]:
!pip3 install minisat

  Running setup.py bdist_wheel for minisat ... done
  Stored in directory: /root/.cache/pip/wheels/98/39/8c/ef1ae79cfc6d117d1ac626bd50c4fd3ba7af340be9a68b3d81
Successfully built minisat


In [13]:
from minisat import *

solver = SatSolver()
a = SatVar()
b = SatVar()
solver.append((-a, -b))
solver.append((-a, b))
solver.append((a, -b))
#solver.append((a, b))
satisfy = solver.solve()

solver.view()
if satisfy:
    print(solver[a], solver[b], solver[c])

NameError: name 'SatSolver' is not defined

### pulp
- わかりやすいし、成功していた

In [6]:
!pip3 install pulp

    100% |################################| 40.6MB 32kB/s eta 0:00:01   17% |#####                           | 7.1MB 1.5MB/s eta 0:00:23    29% |#########                       | 12.0MB 976kB/s eta 0:00:30    30% |#########                       | 12.2MB 1.1MB/s eta 0:00:26    32% |##########                      | 13.0MB 821kB/s eta 0:00:34    37% |###########                     | 15.1MB 1.1MB/s eta 0:00:24    41% |#############                   | 16.8MB 1.2MB/s eta 0:00:20    60% |###################             | 24.7MB 1.0MB/s eta 0:00:16    62% |####################            | 25.5MB 1.6MB/s eta 0:00:10    66% |#####################           | 27.0MB 1.0MB/s eta 0:00:13    71% |######################          | 29.1MB 1.1MB/s eta 0:00:11    72% |#######################         | 29.2MB 1.3MB/s eta 0:00:09    76% |########################        | 31.0MB 1.5MB/s eta 0:00:07
  Running setup.py bdist_wheel for amply ... done
  Stored in directory: /root/.cache/pip/wheels/84/18/f

In [10]:
from pulp import *

# 重さ
w = [4, 2, 2, 1, 10]

# 価値
v = [12, 2, 1, 1, 4]

# 制約：15kg
W = 15

r = range(len(w))

# 数理モデル
m = LpProblem(sense=LpMaximize)

# 変数
x = [LpVariable('x%d'%i, cat=LpBinary) for i in r]

# 目的関数
m += lpDot(v, x)

# 制約
m += lpDot(w, x) <= W 
m.solve() 
print('最大価値:{} / 組み合わせ:{}'.format(value(m.objective), [i for i in r if value(x[i]) > 0.5]))


最大価値:17.0 / 組み合わせ:[0, 3, 4]


### ortoolpyのKnapsack
- [参考にした記事](https://dev2prod.site/python/napsack/)で最もおすすめされていた

In [14]:
!pip3 install ortoolpy

  Running setup.py bdist_wheel for ortoolpy ... done
  Stored in directory: /root/.cache/pip/wheels/04/b2/96/f6ef2da1545a11cd91ee4bd9da930d588dde5751ac7e088a74
Successfully built ortoolpy


In [15]:
from ortoolpy import knapsack

# 重さ
w = [4, 2, 2, 1, 10]

# 価値
v = [12, 2, 1, 1, 4]

# 制約：15kg
W = 15

result = knapsack(w, v, W)

print('最大価値:{} / 組み合わせ:{}'.format(result[0], result))

最大価値:17.0 / 組み合わせ:(17.0, [0, 3, 4])


### cvxpy
- `AttributeError: module 'cvxpy' has no attribute 'ECOS_BB'`が発生している。未解決

In [16]:
!pip3 install cvxpy

    100% |################################| 1.0MB 854kB/s ta 0:00:01
    100% |################################| 153kB 1.4MB/s ta 0:00:01
    100% |################################| 215kB 1.5MB/s ta 0:00:01
    100% |################################| 3.6MB 433kB/s ta 0:00:01    16% |#####                           | 573kB 1.8MB/s eta 0:00:02
  Running setup.py bdist_wheel for cvxpy ... done
  Stored in directory: /root/.cache/pip/wheels/3c/b1/d2/90665270884f32548cdd74d6814d818d89ab6cc61766cde3f3
  Running setup.py bdist_wheel for scs ... done
  Stored in directory: /root/.cache/pip/wheels/df/d0/79/37ea880586da03c620ca9ecd5e42adbd86bc6ea84363965c5f
Successfully built cvxpy scs


In [18]:
from cvxpy import *

# 重さ
w = [4, 2, 2, 1, 10]

# 価値
v = [12, 2, 1, 1, 4]

# 制約：15kg
W = 15

# 変数
x = cvxpy.Variable(shape=len(v), boolean=True)

#　目的関数
objective = cvxpy.Maximize(v * x)

# 制約
constraints = [W >= w * x]

prob = cvxpy.Problem(objective, constraints)

prob.solve(solver=cvxpy.ECOS_BB)

print('最大価値:{} / 組み合わせ:{}'.format(round(prob.value, 0), [i for i in range(len(v)) if round(x.value[i], 0) == 1]))

/usr/local/lib/python3.6/dist-packages/cvxpy/expressions/expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
/usr/local/lib/python3.6/dist-packages/cvxpy/expressions/expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


AttributeError: module 'cvxpy' has no attribute 'ECOS_BB'

In [20]:
#!pip3 install z3-solver

    100% |################################| 29.6MB 58kB/s eta 0:00:01  1% |                                | 409kB 189kB/s eta 0:02:34    1% |                                | 430kB 179kB/s eta 0:02:43    9% |##                              | 2.7MB 1.8MB/s eta 0:00:15    9% |###                             | 2.9MB 1.9MB/s eta 0:00:15    23% |#######                         | 7.0MB 2.5MB/s eta 0:00:10    42% |#############                   | 12.7MB 1.9MB/s eta 0:00:09    49% |###############                 | 14.7MB 2.0MB/s eta 0:00:08    54% |#################               | 16.2MB 2.5MB/s eta 0:00:06    78% |#########################       | 23.1MB 2.2MB/s eta 0:00:03    78% |#########################       | 23.2MB 2.0MB/s eta 0:00:04    88% |############################    | 26.2MB 2.2MB/s eta 0:00:02    91% |#############################   | 27.1MB 1.4MB/s eta 0:00:02


In [ ]:
ys = BoolVector("y", 5)